<a href="https://colab.research.google.com/github/martindevoto/machine-learning-notebooks-personal/blob/main/Intro_Haystack_pt_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Long-Form Question Answering

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepset-ai/haystack/blob/master/tutorials/Tutorial12_LFQA.ipynb)

In [ ]:
# Make sure you have a GPU running
!nvidia-smi

Wed Feb  9 23:49:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss]

     |████████████████████████████████| 2.1 MB 15.4 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-dgok6z1w/farm-haystack_e207f6e0cfd94964a086c625c4303027
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-dgok6z1w/farm-haystack_e207f6e0cfd94964a086c625c4303027
  Resolved https://github.com/deepset-ai/haystack.git to commit 1bdd1f48fd8972128955a75871bc8b8033b3e0f5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.7/359.7 KB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 58.8 MB/s eta 0:00:00
  Preparing meta

In [ ]:
from haystack.utils import (convert_files_to_dicts, fetch_archive_from_http,
                            clean_wiki_text)
from haystack.nodes import Seq2SeqGenerator

### Document Store

FAISS is a library for efficient similarity search on a cluster of dense vectors.
The `FAISSDocumentStore` uses a SQL(SQLite in-memory be default) database under-the-hood
to store the document text and other meta data. The vector embeddings of the text are
indexed on a FAISS Index that later is queried for searching answers.
The default flavour of FAISSDocumentStore is "Flat" but can also be set to "HNSW" for
faster search at the expense of some accuracy. Just set the faiss_index_factor_str argument in the constructor.
For more info on which suits your use case: https://github.com/facebookresearch/faiss/wiki/Guidelines-to-choose-an-index

In [ ]:
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(embedding_dim=128, 
                                    faiss_index_factory_str='Flat')

### Cleaning & indexing documents

Similarly to the previous tutorials, we download, convert and index some Game of Thrones articles to our DocumentStore

In [ ]:
# Let's first get some files that we want to use
doc_dir = "data/article_txt_got"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# Convert files to dicts
dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(dicts)

INFO - haystack.utils.import_utils -  Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip to `data/article_txt_got`
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/488_Brienne_of_Tarth.txt
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/28_Jorah_Mormont.txt
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/449_The_Pointy_End.txt
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/77_Game_of_Thrones_Ascent.txt
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/511_After_the_Thrones.txt
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/454_Music_of_Game_of_Thrones.txt
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/367_Gregor_Clegane.txt
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/515_The_Door__Game_of_Thrones_.txt
INFO - haystack.utils.preprocessing

Writing Documents:   0%|          | 0/2497 [00:00<?, ?it/s]

### Initalize Retriever and Reader/Generator

#### Retriever

**Here:** We use a `RetribertRetriever` and we invoke `update_embeddings` to index the embeddings of documents in the `FAISSDocumentStore`



In [ ]:
from haystack.nodes import EmbeddingRetriever

retriever = EmbeddingRetriever(
    document_store=document_store,
     embedding_model='yjernite/retribert-base-uncased',
     model_format='retribert'
)

document_store.update_embeddings(retriever)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.nodes.retriever.dense -  Init retriever using embeddings of model yjernite/retribert-base-uncased


Downloading:   0%|          | 0.00/487 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/310M [00:00<?, ?B/s]

Some weights of RetriBertModel were not initialized from the model checkpoint at yjernite/retribert-base-uncased and are newly initialized: ['bert_query.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO - haystack.document_stores.faiss -  Updating embeddings for 2357 docs...


Updating Embedding:   0%|          | 0/2357 [00:00<?, ? docs/s]

Creating Embeddings:   0%|          | 0/74 [00:00<?, ? Batches/s]

Before we blindly use the `RetribertRetriever` let's empirically test it to make sure a simple search indeed finds the relevant documents.

In [ ]:
from haystack.utils import print_documents
from haystack.pipelines import DocumentSearchPipeline

p_retrieval = DocumentSearchPipeline(retriever)
res = p_retrieval.run(query="Tell me something about Arya Stark?", params={"Retriever": {"top_k": 10}})
print_documents(res, max_text_len=512)


Query: Tell me something about Arya Stark?

{   'content': '\n'
               '=== Arya Stark ===\n'
               'Arya Stark is the third child and younger daughter of Eddard '
               'and Catelyn Stark. She serves as a POV character for 33 '
               "chapters throughout ''A Game of Thrones'', ''A Clash of "
               "Kings'', ''A Storm of Swords'', ''A Feast for Crows'', and ''A "
               "Dance with Dragons''. So far she is the only character to "
               'appear in all 5 books as a POV character.\n'
               'In the HBO television adaptation, she is portrayed by Maisie '
               'Williams.',
    'name': '30_List_of_A_Song_of_Ice_and_Fire_characters.txt'}

{   'content': '\n'
               '=== Background ===\n'
               'Arya is the third child and younger daughter of Eddard and '
               'Catelyn Stark and is nine years old at the beginning of the '
               'book series.  She has five siblings: an older broth

#### Reader/Generator

Similar to previous Tutorials we now initalize our reader/generator.

Here we use a `Seq2SeqGenerator` with the *yjernite/bart_eli5* model (see: https://huggingface.co/yjernite/bart_eli5)



In [ ]:
generator = Seq2SeqGenerator(model_name_or_path='yjernite/bart_eli5')

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0


Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

### Pipeline

With a Haystack `Pipeline` you can stick together your building blocks to a search pipeline.
Under the hood, `Pipelines` are Directed Acyclic Graphs (DAGs) that you can easily customize for your own use cases.
To speed things up, Haystack also comes with a few predefined Pipelines. One of them is the `GenerativeQAPipeline` that combines a retriever and a reader/generator to answer our questions.
You can learn more about `Pipelines` in the [docs](https://haystack.deepset.ai/docs/latest/pipelinesmd).

In [ ]:
from haystack.pipelines import GenerativeQAPipeline

pipe = GenerativeQAPipeline(generator, retriever)

In [ ]:
pipe.run(
    query="Why did Arya Stark's character get portrayed in a television adaptation?", params={"Retriever": {"top_k": 1}}
)

{'answers': [<Answer {'answer': " Because it's a TV show, and TV shows are usually written by the same people who wrote the book.", 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_id': None, 'meta': {'doc_ids': ['dd0b01f4aee992b812874ee52dbad39e'], 'doc_scores': [0.5549765288619116], 'content': ["\n=== Television series ===\nArya Stark is portrayed by English actress Maisie Williams in the television adaption of the book series, this being Williams' first role as an actress. Williams was chosen from among 300 actresses across England.\nMaisie Williams plays the role of Arya Stark in the television series."], 'titles': ['43_Arya_Stark.txt']}}>],
 'documents': [<Document: {'content': "\n=== Television series ===\nArya Stark is portrayed by English actress Maisie Williams in the television adaption of the book series, this being Williams' first role as an actress. Williams was chosen from among 300 actresses across E

In [ ]:
pipe.run(query="What kind of character does Arya Stark play?", params={"Retriever": {"top_k": 1}})

{'answers': [<Answer {'answer': " Arya Stark is a character in George R.R. Martin's *A Song of Ice and Fire* epic fantasy novel series. She is a prominent point of view in the novels with the third most viewpoint chapters, and is the only viewpoint character in every published book of the series.", 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_id': None, 'meta': {'doc_ids': ['2ee56bdd46dfd30b23f91bcc046456a4'], 'doc_scores': [0.5557039770593725], 'content': ["'''Arya Stark''' is a fictional character in American author George R. R. Martin's ''A Song of Ice and Fire'' epic fantasy novel series.  She is a prominent point of view character in the novels with the third most viewpoint chapters, and is the only viewpoint character to have appeared in every published book of the series.\nIntroduced in 1996's ''A Game of Thrones'', Arya is the third child and younger daughter of Lord Eddard Stark and his wife Lady Catel